In [1]:
import os
from dotenv import dotenv_values
from tqdm.notebook import tqdm
import numpy as np

env = dotenv_values()
txtFilemame = "enwiki-20220601-pages-articles-multistream.xml"
txtPath = os.path.join(env['STORE_PATH'],txtFilemame)
n = 4
histOutputPath = os.path.join(env['STORE_PATH'],"hist89-{}gram.mm".format(n))
print(f"{txtPath=}")
print(f"{histOutputPath=}")

txtPath='/mnt/nvme/btrfs/enwiki-20220601-pages-articles-multistream.xml'
histOutputPath='/mnt/nvme/btrfs/hist89-4gram.mm'


In [2]:
%%time
blocksize = 0xC0000000 * 4 #bytes
fp = np.memmap(histOutputPath, dtype=np.uint32, mode='w+', shape=(4294967296,))
txt = np.memmap(txtPath,mode='r',dtype=np.uint8)
def view(txt,i,offset):
    start = blocksize*i+offset
    end = blocksize*(i+1)+offset
    view = txt[start:end]
    view = txt[start:(start+len(view)-len(view)%4)]
    return view.view(np.uint32)
create_bincount = lambda i,offset : np.bincount(view(txt,i,offset),minlength=0x100000000)
for i, offset in tqdm(np.ndindex((int(len(txt)/blocksize)+1,4))):
    print(f"{i=}, {offset=}")
    %time fp+=create_bincount(i,offset).astype(dtype=np.uint32,copy=False)
!chmod 444 "$histOutputPath"

0it [00:00, ?it/s]

i=0, offset=0
CPU times: user 13.4 s, sys: 25.8 s, total: 39.2 s
Wall time: 1min 14s
i=0, offset=1
CPU times: user 13.2 s, sys: 28.4 s, total: 41.6 s
Wall time: 52.6 s
i=0, offset=2
CPU times: user 13.1 s, sys: 28.6 s, total: 41.7 s
Wall time: 59.5 s
i=0, offset=3
CPU times: user 13 s, sys: 29 s, total: 42 s
Wall time: 1min 3s
i=1, offset=0
CPU times: user 12.8 s, sys: 29.2 s, total: 42 s
Wall time: 49.6 s
i=1, offset=1
CPU times: user 13.2 s, sys: 28.7 s, total: 41.9 s
Wall time: 1min
i=1, offset=2
CPU times: user 13.3 s, sys: 28.4 s, total: 41.7 s
Wall time: 55 s
i=1, offset=3
CPU times: user 12.7 s, sys: 28.1 s, total: 40.8 s
Wall time: 48.6 s
i=2, offset=0
CPU times: user 12.7 s, sys: 27.8 s, total: 40.5 s
Wall time: 50.4 s
i=2, offset=1
CPU times: user 12.8 s, sys: 27.9 s, total: 40.7 s
Wall time: 49 s
i=2, offset=2
CPU times: user 13.2 s, sys: 27.2 s, total: 40.4 s
Wall time: 43.5 s
i=2, offset=3
CPU times: user 12.5 s, sys: 28.5 s, total: 41 s
Wall time: 50.3 s
i=3, offset=0
CPU

In [4]:
n = 4
fp = np.memmap(histOutputPath, dtype=np.uint32, mode='r', shape=(4294967296,))

In [6]:
fp.sum()

89351176749

In [9]:
not_zero = np.where(fp>0)
dict(zip(not_zero[0],fp[not_zero]))

{151587081: 1820348,
 151587082: 649592,
 151587104: 18772,
 151587105: 846,
 151587107: 1,
 151587109: 144,
 151587111: 1512,
 151587112: 7,
 151587113: 2764,
 151587114: 31,
 151587115: 23,
 151587116: 12113,
 151587117: 25172,
 151587118: 495,
 151587119: 85,
 151587120: 2587,
 151587121: 3121,
 151587122: 2607,
 151587123: 1931,
 151587124: 2423,
 151587125: 2658,
 151587126: 2499,
 151587127: 2780,
 151587128: 2417,
 151587129: 1758,
 151587130: 240,
 151587131: 15723,
 151587133: 2841,
 151587134: 2,
 151587135: 65,
 151587137: 119,
 151587138: 1846,
 151587139: 87,
 151587140: 103,
 151587141: 59,
 151587142: 86,
 151587143: 1799,
 151587144: 17,
 151587145: 43,
 151587146: 4,
 151587147: 35,
 151587148: 559,
 151587149: 20,
 151587150: 39,
 151587151: 50,
 151587152: 88,
 151587153: 21,
 151587154: 792,
 151587155: 88,
 151587156: 50,
 151587157: 27,
 151587158: 15,
 151587159: 77,
 151587160: 5,
 151587161: 22,
 151587162: 5,
 151587163: 2,
 151587164: 13,
 151587165: 8081,
 1